# Article Structure Emulation With Marked Keywords

**Задача**: 
1. извлечь из списка маркировок ключевые слова по каждому блоку статьи;
2. выделить для каждой подходящей биграммы предложения, содержащие хотя бы одно маркированное слово;
3. упорядочить их в виде псевдо-статьи (по блокам).

In [1]:
import os
import re

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
import pymorphy2 as pm2

### 1. Data Preprocessing

#### 1.1 Loading corpus

Загрузим датасет - корпус научных статей по физике:

In [2]:
data_directory = '../data/elibrary_physics_15k'
article_id = re.compile(r'_([0-9]+).htm$')

file_contents = []
for filename in tqdm(os.scandir(data_directory), desc='Loading data'):
    if filename.is_file():
        file_name = filename.name
        file_id = int(article_id.search(file_name).groups(1)[0])
        
        with open(filename, "r", encoding='utf-8') as f:
            text = f.read()
            file_contents.append((file_id, text))

print(f'Размер корпуса: {len(file_contents)} статей')

Loading data: 0it [00:00, ?it/s]

Размер корпуса: 15000 статей


#### 1.2 Filtering out non-russian articles

Удалим из корпуса статьи, написанные не на русском языке.

In [3]:
MIN_CYRILLIC_THRESHOLD = 0.2

cyrillic_sym = re.compile(r'[а-яёА-ЯЁ]')

Алгоритм отсеивания статей по языку:
1. подсчитывается кол-во кириллических символов в статье;
2. если соотношение кириллицы >= MIN_CYRILLIC_THRESHOLD -> статья русскоязычная.

In [4]:
rus_articles = []

for article in tqdm(file_contents, desc='Filtering out non-russian articles'):
    rus_count = 0
    for _ in cyrillic_sym.finditer(article[1]):
        rus_count += 1

    if rus_count >= len(article[1]) * MIN_CYRILLIC_THRESHOLD:
        rus_articles.append(article)

print(f'Удалено нерусскоязычных статей: {len(file_contents) - len(rus_articles)}')

Filtering out non-russian articles:   0%|          | 0/15000 [00:00<?, ?it/s]

Удалено нерусскоязычных статей: 4062


In [5]:
articles_count = len(rus_articles)
print(f'Кол-во русскоязычных статей: {articles_count}')

Кол-во русскоязычных статей: 10938


#### 1.3 Arranging lemmatizing procedures

Напишем вспомогательные функции для анализа лемм:

In [6]:
lemmatizer = pm2.MorphAnalyzer(lang='ru')
pm2_cache = {}

In [7]:
def match_bigram(word1_parse, word2_parse):
    """
    Tries to match bigram of w1 and w2 with any of the patterns.
    If succeed, returns a tuple ((w1_norm, w2_norm), pattern).
    """
    word1_pos = word1_parse.tag.POS
    word2_pos = word2_parse.tag.POS

    # failed pattern matching
    if word2_pos != 'NOUN' or word1_pos not in {'NOUN', 'ADJF'}:
        return None

    word1_inflection = (word1_parse.tag.gender, word1_parse.tag.number, word1_parse.tag.case)
    word2_inflection = (word2_parse.tag.gender, word2_parse.tag.number, word2_parse.tag.case)

    # 'adj + noun' pattern
    if (
        word1_pos == 'ADJF' and word2_pos == 'NOUN' 
        and word1_inflection[1] == word2_inflection[1]
        and word1_inflection[2] == word2_inflection[2]
        and (
            word1_inflection[0] == word2_inflection[0]
            or word1_inflection[0] is None and word1_inflection[1] == 'plur'
        )
    ):
        return (word1_parse.normal_form, word2_parse.normal_form), 'прил. + сущ.'

    # 'noun + noun' pattern
    if word1_pos == 'NOUN' and word2_pos == 'NOUN' and word2_inflection[2] == 'gent':
        return (word1_parse.normal_form, word2_parse.normal_form), 'сущ. + сущ.'

    return None

Зададим список стоп-слов:

In [8]:
stop_words = stopwords.words('russian')

stop_words.extend(
    [
        # adjective-like pronouns
        'мой', 'твой', 'ваш', 'наш', 'свой', 'его', 'ее', 'их',
        'тот', 'этот', 'такой', 'таков', 'сей', 'который',
        'весь', 'всякий', 'сам', 'самый', 'каждый', 'любой', 'иной', 'другой',
        'какой', 'каков', 'чей', 'никакой', 'ничей',
        'какой-то', 'какой-либо', 'какой-нибудь', 'некоторый', 'некий',
        # participles
        'соответствующий', 'следующий', 'данный',
        # numerals
        'один',
        # insignificant words
        'друг',
    ]
)

stop_words = set(stop_words)

Зададим список стоп-биграмм:

In [9]:
stop_bigrams = []

stop_bigrams.extend(
    [
        ('крайний', 'мера'), ('сегодняшний', 'день'), ('настоящий', 'время'), ('настоящий', 'работа'),
        ('настоящий', 'статья'), ('точка', 'зрение'), ('первый', 'очередь'), ('последний', 'год'),
    ]
)

stop_bigrams = set(stop_bigrams)

Подходящие слова должны содержать только буквы и дефисы (последние не в начале/конце слова):

In [10]:
def match_unigram(word):
    """Checks if a word is long enough and contains only letters and dashes."""
    if word.startswith('-') or word.endswith('-'):
        return False

    word_dashless = word.replace('-', '')
    return len(word_dashless) >= 3 and word_dashless.isalpha()

Bottleneck лемматизации - обращения к pymorphy2. Сократим кол-во обращений к анализатору, кешируя морфологические разборы токенов:

In [11]:
def get_or_cache_token_parse(lemmatizer, token, cache):
    """Extracts token parse from cache, or adds it to the latter if not found."""
    if token in cache:
        token_parse = cache[token]
    else:
        token_parse = lemmatizer.parse(token)[0]
        cache[token] = token_parse

    return token_parse

### 2. Context Retrieval

Загрузим датасет предложений с посчитанными метриками:

In [12]:
bigram_mi3_df = pd.read_csv('../data/data_frames/bigram_tfidf_mi3_scores.csv', index_col=0)

Введем пороги, по которым будем отличать длинные предложения от средних и коротких:

In [13]:
SHORT_SENT_THRESHOLD = 0.35
MEDIUM_SENT_THRESHOLD = 0.5
LONG_SENT_THRESHOLD = 0.7

SHORT_SENT_MAX_LEN = 10
MEDIUM_SENT_MAX_LEN = 15

Не будем учитывать предложения без "сказуемого" (без семантического анализа их редко когда можно точно определить):

In [14]:
def has_predicate(lemmatizer, cache, tokens, predicates):
    """Checks if the sentence contains a predicate."""
    for token in tokens:
        token_pos = get_or_cache_token_parse(lemmatizer, token, cache).tag.POS
        if token_pos in predicates:
            return True

    return False

In [15]:
def is_definition(bigram, tokens):
    """Checks if the sentence has a structure <... bigram ... - ...>."""
    hyphens = ('-', '−', '–', '—')
    for x in hyphens:
        if x in tokens:
            split_idx = tokens.index(x)
            pre_define = tokens[:split_idx]

            if bigram[0] in pre_define and bigram[1] in pre_define:
                bigram_end_idx = tokens.index(bigram[1])

                if split_idx - bigram_end_idx <= 5:
                    return True

    return False

Извлечем предложения-контексты:

In [16]:
article_info = dict(rus_articles)

In [17]:
bigram_desc_mi3 = bigram_mi3_df.to_dict('records')
predicates = {'INFN', 'VERB', 'PRTS'}

bigram_contexts = {}
for bigram_info in tqdm(bigram_desc_mi3, desc='Extracting bigram contexts'):
    unigram_1 = bigram_info['unigram_1']
    unigram_2 = bigram_info['unigram_2']
    bigram_name = f'{unigram_1}_{unigram_2}'
    bigram_articles = eval(bigram_info['articles_id_list'])

    matching_sentences = []
    for article_id in bigram_articles:
        sentences = sent_tokenize(article_info[article_id], language='russian')

        for sentence_idx, sentence in enumerate(sentences):
            # sentence must start with a capital letter
            if not sentence[0].isupper():
                continue

            tokens = word_tokenize(sentence.lower(), language='russian')
        
            standardized_tokens = []
            for token in tokens:
                if match_unigram(token):
                    standardized_tokens.append(token)

            for i in range(len(standardized_tokens) - 1):
                token1, token2 = standardized_tokens[i], standardized_tokens[i + 1]
        
                token1_parse = get_or_cache_token_parse(lemmatizer, token1, pm2_cache)
                token2_parse = get_or_cache_token_parse(lemmatizer, token2, pm2_cache)

                # bigram must appear before the predicate
                if token1_parse.tag.POS in predicates or token2_parse.tag.POS in predicates:
                    break
                
                if token1_parse.normal_form in stop_words or token2_parse.normal_form in stop_words:
                    continue
        
                bigram_data = match_bigram(token1_parse, token2_parse)
                if bigram_data is not None:
                    bigram, bigram_pattern = bigram_data
        
                    if bigram in stop_bigrams:
                        continue

                    if bigram == (unigram_1, unigram_2):
                        sent_len = len(standardized_tokens)
                        
                        # sentence must contain a predicate or meet a definition template
                        if (
                            not has_predicate(lemmatizer, pm2_cache, standardized_tokens, predicates)
                            and not is_definition((token1, token2), tokens)
                        ):
                            break

                        # the closer bigram to the beginning of the sentence, the bigger its weight
                        bigram_weight = 1 - i / sent_len
                        
                        # saving only those contexts where bigram appeared in the beginning
                        if (
                            sent_len <= SHORT_SENT_MAX_LEN and bigram_weight >= SHORT_SENT_THRESHOLD
                            or sent_len > SHORT_SENT_MAX_LEN and sent_len <= MEDIUM_SENT_MAX_LEN and bigram_weight >= MEDIUM_SENT_THRESHOLD
                            or sent_len > MEDIUM_SENT_MAX_LEN and bigram_weight >= LONG_SENT_THRESHOLD
                        ):
                            matching_sentences.append(sentence)

                        # bigram found -> stop searching
                        break

    # saving extracted bigram contexts
    if matching_sentences:
        bigram_contexts[bigram_name] = matching_sentences

Extracting bigram contexts:   0%|          | 0/211 [00:00<?, ?it/s]

### 3. Forming Article Skeleton

Построим скелет статей по каждой подходящей биграмме.

#### 3.1 Markings Extraction

Для начала, извлечем слова-маркеры:

In [18]:
markers_file = '../data/relation_markers/markers.txt'

In [19]:
block_markers = {}

with open(markers_file, 'r') as f:
    while (line := f.readline()) != '\n':
        line = line.strip()

        # block headline
        if line[0] == '*' and line[1] != '*':
            cur_block = line[1:]
            block_markers[cur_block] = []
        # block separator
        elif line[0] == '*' and line[1] == '*':
            continue
        # marker word
        else:
            block_markers[cur_block].append(line)

In [20]:
block_markers['Структура']

['содержат',
 'компонент',
 'часть',
 'входит в состав',
 'структура',
 'конструкция',
 'включить',
 'включать',
 'состоять',
 'модуль',
 'элемент',
 'принадлежать',
 'состав',
 'структура']

#### 3.2 Sentence Classification

Пройдемся по всем предложениям биграмм и классифицируем их по наличию слов-маркеров:

In [21]:
result_directory = '../data/article_skeletons'

In [22]:
for bigram, contexts in tqdm(bigram_contexts.items(), desc='Creating article skeletons'):
    # classifying sentences
    topic_sentences = defaultdict(list)

    for sentence in contexts:
        is_included = False
        tokens = set(word_tokenize(sentence.lower(), language='russian'))

        for topic, markers in block_markers.items():
            for marker in markers:
                if marker in tokens:
                    topic_sentences[topic].append(sentence)
                    is_included = True
                    break

            if is_included:
                break

    is_empty_skeleton = True
    for content in topic_sentences.values():
        if content:
            is_empty_skeleton = False
            break

    # saving only non-empty results
    if not is_empty_skeleton:
        file_name = f'{result_directory}/{bigram}.txt'
        with open(file_name, 'w') as f:
            f.write(f'{bigram} — термин физики.\n')

            for topic in block_markers:
                sentences = topic_sentences[topic]

                if sentences:
                    f.write('\n')
                    f.write(f'{topic}\n')
                    f.write('*' * 40)
                    f.write('\n')

                    for sentence in sentences:
                        f.write(sentence)
                        f.write('\n')

Creating article skeletons:   0%|          | 0/208 [00:00<?, ?it/s]

### 4. Conclusion

#### 4.1 Notes

- **наихудшее качество** классификации: **определения** и **структура**. В случае определений, слово "это" особенно больно портит картину, т.к. чаще всего оно употребляется в контекстах "это связано с...", "это обстоятельство помогло..." (т.е. это как местоимение-прилагательное). Кажется, маркеров отдельных слов/пар недостаточно для +- адекатной генерации этих 2 блоков - тут могут пригодиться маркеры-шаблоны ("В данной работе рассматривается ...", "... - это ...", т.е. целые синтаксические конструкции как маркеры);
- **наилучшее качество** классификации: **дополнительные сведения**. На удивление, относительно часто выдается неплохая доп. справка по термину, т.е. этот блок более-менее можно составлять чисто на маркерах.
- поскольку каждое предложение учитывал **ровно 1 раз**, то логично, что в первые категории (определения и структура) **попадает большинство** (возможно, и это тоже добаляет шума в выдачу). Возможно, в предложении описывалось использование термина, но из-за слова "это" оно неправильно попало в определения (просто потому что первыми проверялись маркеры именно этого блока). Эту проблему не решить без семантического анализа контекста;
- сами биграммы при записи в 1 строку файлов пока оставлял в виде "норм-форма_норм-форма", если нужно точно восстановить согласованность, то сделаю (правда, это не так просто);

#### 4.2 Improvements

- **TO-DO:**
    1. ;
    2. ;
    3. .